# NOTEBOOK NAME
Created by: Thomas Durand-Texte, 2023

# Import des packages et données
## import des packages

In [ ]:
import os

import pickle
import time

import pandas as pd
import numpy as np
# import dask as dd
pd.set_option("display.max_columns", 200)
import datetime as dt
import scipy.stats as st

from sklearn import model_selection, metrics, preprocessing, linear_model, dummy
from sklearn import svm

import shap
shap.initjs()

import matplotlib.pyplot as plt
import seaborn as sns

cm = 1./2.54

## Paramètres graphiques et fonctions utiles

In [ ]:
import subprocess

white_font = True
def set_theme( white_font=True ):
    """ set_theme( white_font=True ) """
    if white_font: wht, grey, blck = '0.84' , '0.5', 'k'
    else: wht, grey, blck = 'k', '0.5', '0.84'
    rc = { 'figure.facecolor':(0.118,)*3,
            'axes.labelcolor':wht,
            'axes.edgecolor':wht,
            'axes.facecolor':(0,0,0,0),
            'text.color':'white',
            'text.usetex':False,
            'text.latex.preamble':r'\usepackage[cm]{sfmath} \usepackage{amsmath}' ,
            'font.family': 'sans-serif' ,
            'font.sans-serif': 'DejaVu Sans' ,
            'xtick.color':wht,
            'ytick.color':wht,
            "axes.grid" : True,
            "grid.color": (0.7,)*3,
            "grid.linewidth": 0.4,
            "grid.linestyle": (10,5),
            'legend.edgecolor':'0.2',
            'legend.facecolor':(0.2,0.2,0.2,0.6),
            # 'legend.framealpha':'0.6',
            'pdf.fonttype':42,
            'savefig.format':'pdf',
            'savefig.transparent':True,
            'figure.dpi':150, # for better agreemet figsize vs real size
        }

    sns.set_theme( 'notebook' , rc=rc )
    return


def make_folder( path_folder ):
    path_folder = path_folder.__str__()
    try:
        if os.path.isdir( path_folder ) : return
        os.makedirs(path_folder)
    except OSError:
        pass
    return

def concat_folders(*args, **kwargs):
    """ concat_folders(*args, **kwargs)
        concatenate folders in args (strings) """
    sPath = ''
    for arg in args:
        if arg == '..': sPath = sPath[:sPath[:-1].rfind(os.sep)+1]
        else: sPath += arg
        if sPath[-1] != os.sep: sPath += os.sep
    return sPath

class Path(object):
    """ Path( s_in='', s_lim=None)
        create a path to the string s_in (default is current path)
        and stops after s_lim """
    n_Path = 0
    def __init__(self, s_in='', s_lim=None):
        """docstring."""
        if s_in == '': s_in = os.getcwd()
        if not s_lim is None:
            if s_lim in s_in:
                s_in = s_in[ :s_in.index( s_lim ) + len(s_lim) ]
        self.sPath = concat_folders(s_in)
        self.N = Path.n_Path
        Path.n_Path += 1

    def __add__(self, other):
        """ Path + str : return str """
        if isinstance(other, str): return self.sPath + other

    def __truediv__(self, other):
        """ Path / str : return path concatenated"""
        if isinstance(other, str): return Path(concat_folders(self.sPath, other))

    def __invert__(self):
        """ ~Path : return str of the path """
        return self.sPath

    def __str__(self):
        """ __str__ return str of the path """
        return self.sPath
    # __str__ #

    def makedir( self ):
        return make_folder( self )


def gs_opt( filename ):
    """ otpimisation of a pdf file with gosthscript """
    filenameTmp = filename.replace('.pdf', '') + '_tmp.pdf'
    gs = ['gs',
            '-sDEVICE=pdfwrite',
            '-dEmbedAllFonts=true',
            '-dSubsetFonts=true',             # Create font subsets (default)
            '-dPDFSETTINGS=/prepress',        # Image resolution
            '-dDetectDuplicateImages=true',   # Embeds images used multiple times only once
            '-dCompressFonts=true',           # Compress fonts in the output (default)
            '-dNOPAUSE',                      # No pause after each image
            '-dQUIET',                        # Suppress output
            '-dBATCH',                        # Automatically exit
            '-sOutputFile='+filenameTmp,      # Save to temporary output
            filename]                         # Input file

    subprocess.run(gs)                                      # Create temporary file
    subprocess.run( 'rm -f ' + filename, shell=True)            # Delete input file
    subprocess.run( 'mv -f ' + filenameTmp + " " + filename, shell=True) # Rename temporary to input file

def savefig( fig, savename, **kwargs ):
    """ savefig( fig, savename, **kwargs )
        Saves a figure with kwargs (fig.savefig( savename, **kwargs) ).
        A check is done first to determine if a folder has to be created according to savename.
        Finally, if the file is saved as .pdf, gosthscript optimisation is performed. """
    if os.sep in savename: make_folder( savename[:savename.rindex(os.sep)] )
    fig.savefig( savename, **kwargs )
    savename += '.pdf'
    if os.path.isfile( savename ): gs_opt( savename )


def image_size_from_width_and_shape( width: float, shape: tuple, ymargin=0. ):
    """ return tuple (width, height) corresponding to image shape """
    return width, width*shape[0]/shape[1]+ymargin

def image_size_from_height_and_shape( height: float, shape: tuple, xmargin=0. ):
    """ return tuple (width, height) corresponding to image shape """
    return height*shape[1]/shape[0]+xmargin, height


set_theme()
del set_theme

## Chargement des données

Affichage de l'arborescence

In [ ]:
def print_listdir( path=None, level=0, exclude=[] ) :
    suffix = ''
    if level > 0:
        suffix = ' |-'* level
    vals = os.listdir( path )
    vals.sort()
    if path is None:
        path = ''
    for val in vals:
        if val in exclude: continue
        print( suffix, val)
        if os.path.isdir( path + val):
            print_listdir( path + val + '/', level+1 )

print_listdir( exclude=['.venv', 'ressources'] )

Chargement / compression des données

In [ ]:
path = 'data/cleaned/'
filename = '2016_Building_Energy_Benchmarking'
compression = 'gzip'

df = pd.read_pickle( r'{:}{:}.pkl'.format(path, filename), compression=compression )

filename = 'other_data.pkl'
with open( path + filename , 'rb' ) as file:
    features_1,features_2, features_3,features_4, \
    PropertyUseTypes, \
    X_propotion_PropertyUseTypeGFA, X_ParkingGFA,\
    X_propotion_energy = pickle.load( file )

In [ ]:
print('features_1:\n', features_1)
print('\nfeatures_2:\n', features_2)
print('\nfeatures_3:\n', features_3)
print('\nfeatures_4:\n', features_4)

target = features_2[2]
y = df[target].values

# y /= df['PropertyGFABuilding(s)'].values

y_add_transform = (1.-y.min() )
y = np.log( y + y_add_transform )

str_out = f'log( {target} )'

i_nan = np.isnan( y.ravel() )
for Xi in [X_propotion_PropertyUseTypeGFA, X_ParkingGFA, X_propotion_energy]:
    i_nan = i_nan & ( np.isnan( Xi ).reshape(y.size, -1).max(1) )
print( '\nNumber of NaN: {:}'.format( i_nan.sum() ) )

In [ ]:
kurtosis = st.kurtosis( y )
skew = st.skew( y )

fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.hist( y, bins=30 )
ax.set_xlabel( f'log( {target} )' )
ax.set_title( f'skweness: {skew:.3f}, kurtosis: {kurtosis:.3f}')

y2 = y[(y>1) & (y<7)]
kurtosis = st.kurtosis( y2 )
skew = st.skew( y2 )

fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.hist( y2, bins=30 )
ax.set_xlabel( str_out )
ax.set_title( f'skweness: {skew:.3f}, kurtosis: {kurtosis:.3f}')


***
# Definitions

In [ ]:
def train_model( model_name, model, param_grid, score='r2' ): 
    # Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
    modelCV = model_selection.GridSearchCV(
        model, # modèle
        param_grid,     # hyperparamètres à tester
        cv=5,           # nombre de folds de validation croisée
        scoring=score,   # score à optimiser
        n_jobs=-1
    )

    t0 = time.time()
    modelCV.fit(X_train, y_train)
    model_to_results( model_name, modelCV, X_test, time.time()-t0 )

    return modelCV

def log_transfrom( x ):
    return np.log( x + ( 1. - min([0, x.min()]) ) )

def print_coefs( coefs, features ):
    n_features_0 = len( features )

    print('coefficients linear:')
    print(coefs[:n_features_0])

    if len( coefs ) > n_features_0:
        print('coefficients non-linear:')
        print(coefs[n_features_0:])

    print('linear coefs == 0 for :') #, np.array(features)[ coefs[:n_features_0+1] == 0. ])
    print('[', end='')
    for feature in np.array(features)[ coefs[:n_features_0] == 0. ]:
        print( f"'{feature}', ", end='')
    print(']')

    if len( coefs ) > n_features_0+1:
        print('non-linear coefs == 0 for :', np.array(features[1:])[ coefs[n_features_0+1:] == 0. ])

def histogram_per_coef( X, coefs, features ):
    y = (X @ coefs.reshape(-1,1)).ravel()
    for xi, coef, feature in zip( X.transpose(), coefs, features):
        fig, ax = plt.subplots( figsize=(12*cm,8*cm))
        ax.hist( xi*(coef*100)/y, bins=50 )
        ax.set_ylabel('count')
        ax.set_xlabel('influence sur la cible (%)')
        ax.set_title(feature)

In [ ]:
X_GFA_building = df['PropertyGFABuilding(s)'].values.reshape(-1,1)
X_GFA_parking = X_ParkingGFA.reshape(-1,1)

if True:
    key = ['Neighborhood', 'ZipCode'][0]
    df[key] = df[key].astype('category')
    print( df[key].cat.categories )
    features_location = df[key].cat.categories.tolist()
    X_location = np.zeros( (len(df), len(df[key].cat.categories) ) )
    X_location[range(len(df)), df[key].cat.codes] = X_GFA_building.ravel()
else:
    features_location = ['Longitude', 'Latitude']
    X_location = df[features_location].values
    if True:
        X_location = np.hstack( (X_location, X_location**2) )
        features_location += ['Longitude**2', 'Latitude**2']
    X_location *= X_GFA_building.reshape(-1,1)


# for verification
# for i in range(100):
#     print(i, df['Neighborhood'].cat.codes.values[i],  X_neighborhood[i] )

In [ ]:
# X = np.hstack ([X_ParkingGFA.reshape(-1,1)] + [ Xi.reshape(-1,1) * X_propotion_PropertyUseTypeGFA for Xi in X_propotion_energy.transpose()])

# X_propotion_PropertyUseTypeGFA = log_transfrom( X_propotion_PropertyUseTypeGFA )
X_propotion_energy = log_transfrom( X_propotion_energy )

X_GFA_building = log_transfrom( X_GFA_building )
X_GFA_parking = log_transfrom( X_GFA_parking )

df['NumberofBuildings'] = log_transfrom( df['NumberofBuildings'] )
df['NumberofFloors'] = log_transfrom( df['NumberofFloors'] )
# df['YearBuilt'] = log_transfrom( df['YearBuilt'] )
# df['YearBuilt**2'] = df['YearBuilt'].values**2

# X = np.hstack ([X_GFA_parking] + [ Xi.reshape(-1,1) * X_GFA for Xi in X_propotion_energy.transpose()])
X = np.hstack ( [X_GFA_parking]
    + [X_GFA_building] + [X_propotion_PropertyUseTypeGFA] # * X_GFA_building.reshape(-1,1)]
    + [X_location]
    # + [df['Neighborhood'].astype('category').cat.codes.values.reshape(-1,1)]
    + [X_propotion_energy] + [df[['NumberofBuildings', 'NumberofFloors','YearBuilt', 'Outlier']].values] )

features_X = ['GFA_parking', 'GFA_building'] \
    + features_location \
    + PropertyUseTypes.tolist() \
    + features_3 + ['NumberofBuildings', 'NumberofFloors','YearBuilt', 'Outlier']
# X[-4] = np.log( 1 + X[-4] )
# X[-5] = np.log( 1 + X[-5] )

X = np.hstack( (X, X**2))

if False:
    features_zeros = []
    for feature in features_zeros:
        i = features_X.index( feature )
        X = np.delete( X, i, 1 )
        features_X.remove( feature )


X_train, X_test, y_train, y_test = model_selection.train_test_split( X, y, random_state=0)
scaler = preprocessing.StandardScaler().fit( X_test )
X_test = scaler.transform( X_test )
X_train = scaler.transform( X_train )


In [ ]:
print('X', X.shape)
print( 'X_train:', X_train.shape )
# display( len( df['Neighborhood'].value_counts() ) )
# df['Neighborhood'].astype('category').cat.codes

In [ ]:
print( 'X_parking:', X_ParkingGFA.shape )
print( 'X_PropertyUseType:', X_propotion_PropertyUseTypeGFA.shape )
print( 'X_propotion_energy:', X_propotion_energy.shape )
print('features_1:\n', features_1)
print('\nfeatures_2:\n', features_2)
print('\nfeatures_3:\n', features_3)
print('\nfeatures_4:\n', features_4)
print('\nfeatures_X:\n', features_X)

In [ ]:
dico_results = {'y_test':y_test}

In [ ]:
def plot_results( label ):
    y_test, y_pred = dico_results['y_test'], dico_results[label]['y_pred']

    argsort = y_test.argsort()
    fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
    ax.plot( y_test[argsort], 'r', label='data' )
    ax.plot( y_pred[argsort], 'bo', markersize=2, label=label )
    ax.legend()
    ax.set_xlabel('sample')

    fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
    ax.plot( y_test, y_pred, 'bo', markersize=2, label=f'prediction {label}' )
    xy_optim = [y_test.min(), y_test.max()]
    ax.plot( xy_optim, xy_optim, 'r', label='optimal fit' )
    ax.legend()
    ax.set_xlabel( 'prediction ' + label)
    imin = y_test.argmin()
    print( y_pred[imin], y_test[imin] )

    ax.set_xlabel( str_out )

def model_to_results( name, model, X_test, training_time ):
    dico_results[name] = {'y_pred': model.predict(X_test),
                        'training time': training_time,
                        'model':model }

def print_results( ):
    y_test = dico_results['y_test']
    df_results = pd.DataFrame( index=['training time (sec.)', 'MSE', 'R2 score'])
    for (name, results) in dico_results.items():
        if name == 'y_test' :
            continue
        y_pred = results['y_pred']
        df_results[name] = [results['training time'], 
                            metrics.mean_squared_error(y_test, y_pred),
                            metrics.r2_score(y_test, y_pred) ]
    
    display( df_results )

# Regressor data

In [ ]:
fig, ax = plt.subplots( figsize=(12*cm, 8*cm))
ax.plot( X_propotion_PropertyUseTypeGFA.sum(1) * df['PropertyGFABuilding(s)'], y, 'bo', markersize=2 )
ax.set_xlim([-0.5e6, 2.2e6])
ax.set_ylim([-0.5, 22])

# Dummy Regressor

In [ ]:
model_name = 'dummy median'
t0 = time.time()
dr = dummy.DummyRegressor( strategy='median' ).fit( X_train, y_train )
model_to_results( model_name, dr, X_test, time.time()-t0 )

In [ ]:
dico_results.keys()

In [ ]:
print_results()
plot_results( model_name )

***
# Lasso

In [ ]:
model_name = 'lasso'
alphas_lasso = np.logspace(-5, 0, 100)
t0 = time.time()
model = linear_model.LassoCV( alphas=alphas_lasso, cv=5 ).fit( X_train, y_train )

model_to_results( model_name, model, X_test, time.time()-t0 )
_, coefs_lasso, _ = linear_model.lasso_path( X_train, y_train, alphas=alphas_lasso )

In [ ]:
alpha_lassoCV = model.alpha_
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print( 'alpha:', alpha_lassoCV)

print_results()
plot_results( model_name )

# print( alphas_lasso.shape )
# print(coefs_lasso.shape)
# print(X_train.shape)

# histogram_per_coef( X_train, model.cohistogram_per_coef( X, coefs, features )ef_, features_X )

print_coefs( model.coef_, features_X )
argsort = np.abs(model.coef_).argsort()
print('\nsorted coefs:')
for i in argsort:
    print( f'{features_X[i]}: { model.coef_[i]:.6f}' )

print('\nlowest coefs:')
print('[', end='')
for i in argsort[:-10]:
    print( f"'{features_X[i]}', ", end='')
print(']')

fig, ax = plt.subplots( figsize=(20*cm,12*cm) )
for feature, coefs in zip( features_X, coefs_lasso ):
    ax.semilogx( alphas_lasso, coefs, label=feature)
# ax.semilogx( alphas_lasso, coefs_lasso.transpose() )
ax.plot( [alpha_lassoCV,]*2, [coefs_lasso.min(), coefs_lasso.max()], 'r--' )
# ax.legend()



argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()
ax.set_title('train set')

***
# Elastic net

In [ ]:
# help( linear_model.ElasticNet )

In [ ]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings

In [ ]:
model_name = 'Elastic net'

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'l1_ratio':np.geomspace(0.5, 0.99, 10),
            'alpha':np.logspace(-5, 1, 20)}

model = train_model( model_name, linear_model.ElasticNet(random_state=0), param_grid )

In [ ]:
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(model.best_params_)

print_coefs( model.best_estimator_.coef_, features_X )

In [ ]:
print_results()
plot_results( model_name )

print( 'train MSE: {:.3f}'.format( metrics.mean_squared_error( y_train, model.predict(X_train) ) ) )
print( 'train r2: {:.3f}'.format( metrics.r2_score( y_train, model.predict(X_train) ) ) )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()
ax.set_title('train set')

if False:
    fig, ax = plt.subplots( figsize=(12*cm,8*cm))
    ax.plot( [y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r' )
    ax.plot( y_train, model.predict( X_train) , 'bo', markersize=3 )

***
# SVR

In [ ]:
from sklearn.svm import LinearSVR
# help( LinearSVR )

model_name = 'Linear SVR'

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'loss':['epsilon_insensitive', 'squared_epsilon_insensitive'],
            'C':np.logspace(-5, 10, 20)
            }


model = train_model( model_name, LinearSVR( dual=False ), param_grid)

In [ ]:
print_results()
plot_results( model_name )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()

***
# Kernel Ridge

In [ ]:
from sklearn import kernel_ridge

model_name = 'Kernel ridge'

# Fixer les valeurs des hyperparamètres à tester
param_grid = {
            'alpha':np.logspace(-4, -2, 10),
            'gamma':np.logspace(-4, -2, 10),
            }

model = train_model( model_name, kernel_ridge.KernelRidge(kernel='rbf'), param_grid )

In [ ]:
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(model.best_params_)


print_results()
plot_results( model_name )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()

***
# Kernel SVR

In [ ]:
help( svm.SVR )

In [ ]:
model_name = 'Kernel SVR'

# Fixer les valeurs des hyperparamètres à tester
param_grid = {
            'C':np.logspace(0, 4, 10),
            'gamma':np.logspace(-5, -3, 10),
            }

model = train_model( model_name, svm.SVR(kernel='rbf'), param_grid )

In [ ]:
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(model.best_params_)


print_results()
plot_results( model_name )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()

***
# Random Forest

In [ ]:
help( RandomForestRegressor)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# help( RandomForestRegressor )

model_name = 'Random forest'
param_grid = {
    'n_estimators':[300, 400, 500],
    'min_samples_split': [10, 20, 50],
    'max_depth':[None, 5, 10]
}

model = train_model( model_name, RandomForestRegressor(), param_grid )

In [ ]:
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(model.best_params_)


print_results()
plot_results( model_name )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()
ax.set_title('train set')

***
# LightGBM

In [ ]:
import lightgbm as lgb

model_name = 'Light GBM'
param_grid = {
    'n_estimators': [150, 200, 300],
    # 'max_features': [ i  for i in range(1, X.shape[1], 10) ] + [X.shape[1]],
    'subsample': np.arange(0.1, 1.1, 0.1),
    'learning_rate': [0.001, 0.01, 0.1, 1.0],
    'max_depth': np.arange(2, 11, 2),
}

model = train_model( model_name, lgb.LGBMRegressor(), param_grid )

In [ ]:
# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(model.best_params_)


print_results()
plot_results( model_name )

argsort = y_train.argsort()
fig, ax = plt.subplots( figsize=(12*cm,8*cm) )
ax.plot( y_train[argsort], 'r' )
ax.plot( model.predict( X_train )[argsort], 'bo', markersize=2, label=model_name )
ax.legend()
ax.set_title('train set')